In [1]:
import numpy as np
from scipy.integrate import quad
import pygame 

from formula_functions import *
from animation_functions import *

from show_parameters_functions import *
from key_controls_functions import *
from formula_functions import *

from Data_classes.data_processing import DataProcessing

pygame 2.4.0 (SDL 2.26.4, Python 3.9.13)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
dd = DataProcessing()
dd.constants.constants_dict

{'deg': [0, 0],
 't': 1,
 'v': 1,
 'w': 1,
 'k': 0,
 'x': 1,
 'c': [750.0, 400.0],
 'l': 75.0}

In [3]:
def sign_func(x,a,b=None):
    if b is None:
        return (x-a)/abs(E(x-a))
    prod = (x-a)*(x-b)
    return prod/abs(E(prod))

In [6]:
def draw_explanations(data_processing,font_small, t_mth_list):
    
    explanations_layer = data_processing.mode_statuses_dict['Explanations layer'][0]
    explanations_layer.fill((0, 0, 0, 0))

    
    dots_params = data_processing.draw_dots_params
    const_center_point = dots_params[6]
    var_center_point = dots_params[7]

    center_point_width = const_center_point[0] + var_center_point[0]
    center_point_height = const_center_point[1] + var_center_point[1]
    length = data_processing.get_curr_param('l')

    x = np.linspace(-10,10, 3000)
    
    x_1 = np.array([abs(x)*sign_func(x, 1,b=4) for x in x])
 
    y = np.sqrt(x_1)
    
    x = [x_transform(x, center_point_width, length) for x in x]
    y = [y_transform(y, center_point_height, length) for y in y]
    
    for i in range(len(x) -1):

        pygame.draw.circle(explanations_layer, color='red', center=(x[i], y[i]), radius=1)

In [ ]:
def main_loop():

    '''
    Initializations
    
    '''
    
    pygame.init()

    line_color = (185, 185, 185) # Color of the coordinate line
    
    clock = pygame.time.Clock()
    
    
    data_processing = DataProcessing()
    
    # data_processing.constants.text_spacing = 45
    # data_processing.constants.text_spacing = 26
    
    data_processing.constants.text_unit = data_processing.constants.text_spacing / 1.3
    
    font_small = pygame.font.Font(None, data_processing.constants.text_spacing)

    screen_width, screen_height, FPS = data_processing.constants.screen_width, \
                                       data_processing.constants.screen_height, \
                                        data_processing.constants.FPS
    
    # Layers
    win = pygame.display.set_mode((screen_width, screen_height)) # Main animation layer

    # data_processing.reset_dicts()
    

#     data_processing.constants.constants_dict['t'] = 6.62
#     data_processing.constants.constants_dict['x'] = -1.6
#     data_processing.constants.constants_dict['v'] = 0.25
#     data_processing.constants.constants_dict['w'] = 0.29
#     # data_processing.constants.constants_dict['b'] = -3.9
#     data_processing.constants.constants_dict['k'] = 0.8
#     # data_processing.constants.constants_dict['a'] = 59
#     data_processing.variables.variables_dict['l'] = 1296
#     # data_processing.variables.variables_dict['l'] = 700
    
#     data_processing.variables.variables_dict['c'] = [2060, 180]
    
  
    data_processing.constants.constants_dict['t'] = 8.65
    data_processing.constants.constants_dict['x'] = 0.5
    # data_processing.constants.constants_dict['v'] = 0.27
    # data_processing.constants.constants_dict['w'] = 2
    data_processing.variables.variables_dict['b'] = 0
    # data_processing.constants.constants_dict['k'] = 3.5
    # data_processing.variables.variables_dict['a'] = 45
    # data_processing.variables.variables_dict['l']  = 252
    # data_processing.variables.variables_dict['c'] = [160, 10]
    # data_processing.mode_statuses_dict['General solution'][1] = True

    t_mth_aproxim_list = [] # caches the already computed approximation values

    create_background(*data_processing.background_params, font_small)

    draw_line(*data_processing.line_params)
    
    
    draw_derivatives(*data_processing.derivatives_params)

    draw_spiral(*data_processing.spiral_params, font_small)
    
    t_nth_list = calc_y_intersects_t(*data_processing.y_intersects_t_params)

    # Initial drawing of the y-intersection points
    draw_dots(*data_processing.draw_dots_params, t_nth_list, 'Y-intersects')

    show_parameters(data_processing, font_small)
    
    show_mode_statuses(data_processing, font_small)
    
    
    data_processing.blit_layers(win)
    
    pygame.display.update()
 
    running = True
    
    while running:
        
        clock.tick(data_processing.constants.FPS)

        # All updates booleans must be set to False
        data_processing.booleans.reset_to_false()
      
        # Check and handle for key control events
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running = False

            else:
    
                handle_ctrl_commands(event, data_processing)
                
                handle_steps_variables(event, data_processing)
  
                handle_switch_commands(event, data_processing)
                
    
                if data_processing.mode_statuses_dict['Algorithm mode'][1]:
                    t_mth_aproxim_list = handle_algorithm_mode_controls(event, data_processing, t_mth_aproxim_list)
 
                else:

                    handle_shift_key_commands(event, data_processing)

                if data_processing.booleans.update_booleans_dict['is_turn_off'] is not None:
                    
                    data_processing.mode_statuses_dict['Algorithm mode'][0].fill((0, 0, 0,0))
                    data_processing.mode_statuses_dict['Algorithm data mode'][0].fill((0, 0, 0, 0))
                    t_mth_aproxim_list.clear()
                    data_processing.algorithm_vars.reset_dict()
                    data_processing.mode_statuses_dict['Derivatives'][1] = False

        handle_key_commands(data_processing)

        if data_processing.booleans.update_booleans_dict['update_screen']:

            if data_processing.mode_statuses_dict['Algorithm mode'][1]:
                
                data_processing.mode_statuses_dict['Derivatives'][1] = False
                
            draw_derivatives(*data_processing.derivatives_params)
            
            if data_processing.booleans.update_booleans_dict['shift_coords'] \
                or data_processing.booleans.update_booleans_dict['reset_background']:
               
                # Update coordinate system
                create_background(*data_processing.background_params, font_small)

            
            if data_processing.booleans.update_booleans_dict['update_spiral'] \
                or data_processing.booleans.update_booleans_dict['is_t_diagram_change']:
                draw_spiral(*data_processing.spiral_params, font_small)

            if data_processing.booleans.update_booleans_dict['update_line']:         
                
                draw_line(*data_processing.line_params)

            # A list contains the exact t-parameters for the intersection points of the spiral with y-axis
            t_nth_list = calc_y_intersects_t(*data_processing.y_intersects_t_params)
            
            # Draw current y intersection points if y-intersects mode is `on`
            if data_processing.mode_statuses_dict['Y-intersects'][1]:
             
                draw_dots(*data_processing.draw_dots_params, t_nth_list, 'Y-intersects')

            # A list contains the exact t-parameters for the intersection points of the spiral with the vertical line
            t_mth_list = calc_line_intersections_t(*data_processing.line_intersections_t_params, t_nth_list)
         
            # Draw current line intersection points if line-intersects mode is `on`
            if data_processing.mode_statuses_dict['Line-intersects'][1]:
                draw_dots(*data_processing.draw_dots_params, t_mth_list, 'Line-intersects')
      
            # draw_circle_and_vector(data_processing, font_small)
                
            # Show steps variables if the steps change mode is `on`    
            show_steps_variables(*data_processing.show_steps_params, font_small)
            
            # Show the current statuses of the spiral and line parameters
            show_parameters(data_processing, font_small)  
            
            # Show the current statuses of the mode parameters
            show_mode_statuses(data_processing, font_small)
            
            # show_circle_mode_parameters(data_processing, font_small)

            # Show the algorithm steps towards the interesection point
            if data_processing.mode_statuses_dict['Algorithm mode'][1]:
                
                # print('List before: ', data_processing.x_deriv_list)
                t_mth_aproxim_list, data_processing.algorithm_vars.algorithm_vars_dict['total_n'] = \
                draw_algorithm_steps(*data_processing.draw_algorithm_params, t_nth_list, t_mth_aproxim_list)

                x, y = 'None', 'None'
                if t_nth_list:
                    nth_t = t_nth_list[data_processing.algorithm_vars.algorithm_vars_dict['n']]
                else:
                    nth_t = 0
                mth_t = 0
         
                if len(t_mth_aproxim_list) > 1:
  
                    mth_t = t_mth_aproxim_list[data_processing.algorithm_vars.algorithm_vars_dict['m']]
            
                    x, y = calc_single_t_aproxim(*data_processing.calc_sigle_t_approxim_params, mth_t, transform=False)

                # Show the current statuses of the m_th aproximation point
                show_algorithm_rows_and_cols(data_processing, nth_t, mth_t,  x, y, font_small)
                
            draw_explanations(data_processing, font_small, t_mth_aproxim_list)
            
            # print('l: ', data_processing.variables.variables_dict['l'])
#             print('l curr param: ', data_processing.get_curr_param('l'))
        
            # print('c: ', data_processing.variables.variables_dict['c'])
            
            
            # Attach the new drawings on the screen    
            data_processing.blit_layers(win)
            
            pygame.display.update()

        
    # Exit
    pygame.quit()
    
main_loop()

C:\Users\lenovo\AppData\Local\Temp\ipykernel_18792\1711361936.py:19: RuntimeWarning: invalid value encountered in sqrt
  y = np.sqrt(x_1)
